# Lab 2b: Eigendecomposition of Stoichiometric Matrices
Fill me in

## Setup, Data and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl");

  Activating project at `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b`
    Updating `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b/Project.toml`
  [336ed68f] + CSV v0.10.15
  [a93c6f00] + DataFrames v1.7.0
  [5789e2e9] + FileIO v1.16.6
  [cd3eb016] + HTTP v1.10.15
  [033835bb] + JLD2 v0.5.11
  [682c06a0] + JSON v0.21.4
  [b27032c2] ~ LibCURL ⇒ v0.6.4
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Documents/GitHub/CHEME-5820-Labs-Spring-2025/labs/week-2/L2b/Manifest.toml`
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [944b1d66] + CodecZlib v0.7.6
  [34da2185] + Compat v4.16.0
  [f0e56b4a] + ConcurrentUtilities v2.4.3
  [a8cc5b0e] + Crayons v4.1.1
  [9a962f9c] + DataAPI v1.16.0
  [a93c6f00] + DataFrames v1.7.0
  [864edb3b] + DataStructures v0.18.20
  [e2d170a0] + DataValueInterfaces v1.0.0
  [460bff9d] + ExceptionUnwrapping v0.1.11
  [5789e2e9] + FileIO v1.16.6
  [48062228] + FilePathsBase v0.9.22
  [cd3eb016] + HTTP v1.10.15
  [8

Download a model.

In [2]:
model = let

    # build download endpoint -
    baseurl = "http://bigg.ucsd.edu"; # base url to download model
    modelid = "e_coli_core"; # model id to download
    path_to_saved_model_file = joinpath(_PATH_TO_DATA, "saved-model-$(modelid).jld2");

    # check: do we have a model file saved?
    model = nothing;
    if (isfile(path_to_saved_model_file) == false)
        
        endpoint = MyBiggModelsDownloadModelEndpointModel();
        endpoint.bigg_id = modelid;
        url = build(baseurl, endpoint)
        model = MyBiggModelsDownloadModelEndpointModel(url);

        # Before we move on, save this model for later (so we don't keep hitting the API)
        save(path_to_saved_model_file, Dict("model" => model));
    else
        model = load(path_to_saved_model_file)["model"];
    end
    model; # return the model (either saved, or downloaded)
end

Dict{String, Any} with 6 entries:
  "metabolites"  => Any[Dict{String, Any}("compartment"=>"e", "name"=>"D-Glucos…
  "id"           => "e_coli_core"
  "compartments" => Dict{String, Any}("c"=>"cytosol", "e"=>"extracellular space…
  "reactions"    => Any[Dict{String, Any}("name"=>"Phosphofructokinase", "metab…
  "version"      => "1"
  "genes"        => Any[Dict{String, Any}("name"=>"adhE", "id"=>"b1241", "notes…

Next, let's build a stoichiometric matrix.

In [3]:
model["metabolites"][1] # example metabolite record

Dict{String, Any} with 7 entries:
  "compartment" => "e"
  "name"        => "D-Glucose"
  "formula"     => "C6H12O6"
  "id"          => "glc__D_e"
  "charge"      => 0
  "notes"       => Dict{String, Any}("original_bigg_ids"=>Any["glc_D_e"])
  "annotation"  => Dict{String, Any}("kegg.drug"=>Any["D00009"], "sabiork"=>Any…

In [4]:
model["reactions"][1] # example reaction record

Dict{String, Any} with 9 entries:
  "name"               => "Phosphofructokinase"
  "metabolites"        => Dict{String, Any}("adp_c"=>1.0, "atp_c"=>-1.0, "f6p_c…
  "lower_bound"        => 0.0
  "id"                 => "PFK"
  "notes"              => Dict{String, Any}("original_bigg_ids"=>Any["PFK"])
  "gene_reaction_rule" => "b3916 or b1723"
  "upper_bound"        => 1000.0
  "subsystem"          => "Glycolysis/Gluconeogenesis"
  "annotation"         => Dict{String, Any}("bigg.reaction"=>Any["PFK"], "metan…

In [5]:
S = let

    # get some data from the model -
    m = model["metabolites"]; # get list of metabolites
    r = model["reactions"]; # get list of reactions
    number_of_rows = length(m); # how many metabolites do we have? (rows)
    number_of_cols = length(r); # how many reactions do we have? (cols)
    S = zeros(number_of_rows,number_of_cols); # initialize an empty stoichiometric matrix

    # let's build a stm -
    for i ∈ eachindex(m)
        metabolite = m[i]["id"]; # we are checking if this metabolite is in the reaction record
        for j ∈ eachindex(r)
            reaction = r[j];
            if (haskey(reaction["metabolites"], metabolite) == true)
                S[i,j] = reaction["metabolites"][metabolite];
            end
        end
    end
    S;
end;

Binary stoichiometric array.

In [6]:
Ŝ = let

    (m,r) = size(S);
    Ŝ = zeros(m,r);

    for i ∈ 1:m
        for j ∈ 1:r
            if (S[i,j] != 0.0)
                Ŝ[i,j] = 1.0;
            end
        end
    end    
    Ŝ;
end

72×95 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [7]:
i = S[:,1] |> x -> findall(m -> m != 0.0, x);
m = model["metabolites"][i] .|> m -> m["id"]

5-element Vector{String}:
 "h_c"
 "adp_c"
 "atp_c"
 "f6p_c"
 "fdp_c"

In [8]:
A = transpose(S)*S

95×95 Matrix{Float64}:
  5.0   0.0  -1.0   2.0   1.0   1.0  …   0.0   -4.0   0.0  0.0  0.0   0.0
  0.0   4.0   0.0   0.0   0.0   2.0     -1.0    0.0   0.0  0.0  0.0   3.0
 -1.0   0.0   2.0   0.0   0.0   0.0      0.0    0.0   0.0  0.0  0.0   0.0
  2.0   0.0   0.0   4.0   0.0   0.0      0.0    0.0   0.0  0.0  0.0   0.0
  1.0   0.0   0.0   0.0   4.0   1.0      0.0   -4.0   0.0  0.0  0.0   0.0
  1.0   2.0   0.0   0.0   1.0   6.0  …   0.0   -6.0   2.0  0.0  0.0   4.0
  1.0   0.0   0.0   0.0   1.0   1.0      0.0   -7.0   0.0  0.0  0.0   0.0
  0.0   0.0   0.0  -1.0   0.0   0.0      0.0    0.0   0.0  0.0  0.0   0.0
  1.0   0.0   0.0   0.0   1.0   1.0      0.0   -7.0   0.0  0.0  0.0   0.0
  1.0   0.0   0.0   0.0   1.0   2.0      0.0   -6.0   2.0  0.0  0.0   2.0
  0.0   0.0   0.0   0.0   0.0  -1.0  …   0.0    0.0   0.0  0.0  0.0   0.0
  2.0   0.0   0.0   2.0   0.0   0.0      0.0    0.0   0.0  0.0  0.0   0.0
  1.0   0.0   0.0   0.0   2.0   1.0     -1.0   -4.0   0.0  0.0  0.0  -1.0
  ⋮            

In [9]:
(λ, V) = let
    F = eigen(A)
    λ = F.values
    V = F.vectors;
    λ,V
end;

In [10]:
λ

95-element Vector{Float64}:
    -5.43916732781253e-13
    -3.1345029002727397e-13
    -1.4368964136799667e-13
    -2.4609023408405857e-14
    -1.9085291125109742e-14
    -1.605095040435873e-14
    -9.5011692322834e-15
    -9.43459782200966e-15
    -8.218911057864698e-15
    -6.650847310648314e-15
    -6.604102758255699e-15
    -5.7953895714569975e-15
    -5.3013149425851225e-15
     ⋮
     7.179651550389857
     7.981251263708931
     9.303622040338322
    11.409865219522086
    12.527717460707946
    14.051747930083371
    14.948791470915314
    20.036045187910993
    23.11730912834329
    28.587080267156356
   110.07704573669785
 18380.95463382443

In [11]:
imax = argmax(abs.(V[:,92]));

In [12]:
model["reactions"][imax]

Dict{String, Any} with 9 entries:
  "name"               => "Glutamate synthase (NADPH)"
  "metabolites"        => Dict{String, Any}("glu__L_c"=>2.0, "nadph_c"=>-1.0, "…
  "lower_bound"        => 0.0
  "id"                 => "GLUSy"
  "notes"              => Dict{String, Any}("original_bigg_ids"=>Any["GLUSy"])
  "gene_reaction_rule" => "b3212 and b3213"
  "upper_bound"        => 1000.0
  "subsystem"          => "Glutamate Metabolism"
  "annotation"         => Dict{String, Any}("bigg.reaction"=>Any["GLUSy"], "sab…

In [13]:
λ

95-element Vector{Float64}:
    -5.43916732781253e-13
    -3.1345029002727397e-13
    -1.4368964136799667e-13
    -2.4609023408405857e-14
    -1.9085291125109742e-14
    -1.605095040435873e-14
    -9.5011692322834e-15
    -9.43459782200966e-15
    -8.218911057864698e-15
    -6.650847310648314e-15
    -6.604102758255699e-15
    -5.7953895714569975e-15
    -5.3013149425851225e-15
     ⋮
     7.179651550389857
     7.981251263708931
     9.303622040338322
    11.409865219522086
    12.527717460707946
    14.051747930083371
    14.948791470915314
    20.036045187910993
    23.11730912834329
    28.587080267156356
   110.07704573669785
 18380.95463382443

In [14]:
B = S*transpose(S)

72×72 Matrix{Float64}:
  2.0    0.0         0.0     0.0       …   0.0   0.0   0.0         -1.0
  0.0    4.06538    -1.0    -2.73648       0.0   0.0   0.0329853    0.0524185
  0.0   -1.0         2.0     0.0           0.0   0.0   0.0          0.0
  0.0   -2.73648     0.0    32.4174        0.0   0.0   0.637441     1.01299
  0.0    0.0         0.0    -1.0           0.0   0.0   0.0          0.0
  0.0    0.0         0.0     0.0       …   0.0   0.0   0.0          0.0
  0.0   15.2934      1.0   295.545         1.0   0.0   7.71549     12.261
  0.0    0.0         0.0     0.0           0.0   0.0   0.0          0.0
  0.0  -12.2934     -1.0  -298.545         2.0  -2.0  -8.71549    -13.261
  0.0    0.0         0.0    -1.0          -2.0   2.0   0.0          0.0
  0.0    0.0         0.0     0.0       …   0.0   0.0   0.0          0.0
  0.0    0.0         0.0     0.0           0.0   0.0   0.0          0.0
  0.0    0.0         0.0     0.0           0.0   0.0   0.0          0.0
  ⋮                        

In [15]:
(λ̂, V̂) = let
    F = eigen(B)
    λ = F.values
    V = F.vectors;
    λ,V
end;

In [16]:
λ̂

72-element Vector{Float64}:
    -2.5139842438751256e-13
    -1.036257577237626e-15
    -7.912576699525912e-16
    -1.0359315354547157e-16
     8.296695395685386e-13
     0.013482156741456438
     0.11584214049426257
     0.1547500835845582
     0.2003211061826032
     0.29949893101043173
     0.3725051995305418
     0.40158399313598475
     0.4321113939614752
     ⋮
     7.179651550389532
     7.981251263708104
     9.303622040337865
    11.409865219521995
    12.527717460707915
    14.051747930083273
    14.94879147091537
    20.03604518791126
    23.117309128343408
    28.58708026715736
   110.0770457366979
 18380.95463382443

In [17]:
imax = argmax(abs.(V̂[:,70]));

In [18]:
model["metabolites"][imax]

Dict{String, Any} with 7 entries:
  "compartment" => "c"
  "name"        => "Nicotinamide adenine dinucleotide"
  "formula"     => "C21H26N7O14P2"
  "id"          => "nad_c"
  "charge"      => -1
  "notes"       => Dict{String, Any}("original_bigg_ids"=>Any["nad_c"])
  "annotation"  => Dict{String, Any}("kegg.drug"=>Any["D00002"], "sabiork"=>Any…